# <a id='vverh'>Домашнее задание №4 </a>
1. Реализуйте дерево для задачи регрессии. Возьмите за основу дерево, реализованное в методичке, заменив механизм предсказания в листе на взятие среднего значения по выборке, и критерий Джини на дисперсию значений.

не похоже, что дерево понимает, что от него хотят с такими плохими оценками качества, но вроде бы в задании надо только заменить пару вещей, а не оптимизировать модель

[Ячейка с изменениями №1](#10)

[Ячейка с изменениями №2](#11)

In [1]:
import matplotlib.pyplot as plt
import random

from matplotlib.colors import ListedColormap
from sklearn.datasets import make_classification, make_circles
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, plot_tree
from sklearn.metrics import accuracy_score

import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Реализуем класс узла

class Node:
    
    def __init__(self, index, t, true_branch, false_branch):
        self.index = index  # индекс признака, по которому ведется сравнение с порогом в этом узле
        self.t = t  # значение порога
        self.true_branch = true_branch  # поддерево, удовлетворяющее условию в узле
        self.false_branch = false_branch  # поддерево, не удовлетворяющее условию в узле

<a id='10'>ячейка с изменениями 1</a> <br>
Тут я заменил весь предикт на одну формулу, перед этим я посмотрел что вообще происходит внутри, чтобы понять что именно мне нужно считать. у меня есть по сути только дата и метки. дата это про данные (вес и рост в тех данных что есть), а метки это единственное что остаётся <br>
[Мне на верх](#vverh)

In [3]:
# И класс терминального узла (листа)

class Leaf:
    
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        self.prediction = self.predict()
        
    def predict(self):
#         print(self.data, '<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<')
#         print(self.labels, 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx')
#         # подсчет количества объектов разных классов
#         classes = {}  # сформируем словарь "класс: количество объектов"
#         for label in self.labels:
#             if label not in classes:
#                 classes[label] = 0
#             classes[label] += 1
#         print(classes, '+++++++++++++')   
        # найдем класс, количество объектов которого будет максимальным в этом листе и вернем его    
#         prediction = max(classes, key=classes.get)
#         print(self.labels, '2x22x2x2x2x2xx2')
        prediction = sum(self.labels)/len(self.labels)
#         print(prediction, '---------')
        return prediction   

[Мне на верх](#vverh)

In [4]:
# Расчет критерия Джини

def gini(labels):
    #  подсчет количества объектов разных классов
    classes = {}
    for label in labels:
        if label not in classes:
            classes[label] = 0
        classes[label] += 1
    
    #  расчет критерия
    impurity = 1
    for label in classes:
        p = classes[label] / len(labels)
        impurity -= p ** 2
        
    return impurity

In [5]:
def mse(array):
    mean = array.mean()
    return np.mean((array - mean)**2)

In [6]:
# Расчет прироста

def gain(left_labels, right_labels, root_criterion, criterion):

    # доля выборки, ушедшая в левое поддерево
    p = float(left_labels.shape[0]) / (left_labels.shape[0] + right_labels.shape[0])
    
    return root_criterion - p * criterion(left_labels) - (1 - p) * criterion(right_labels)

In [7]:
# Разбиение датасета в узле

def split(data, labels, column_index, t):
    
    left = np.where(data[:, column_index] <= t)
    right = np.where(data[:, column_index] > t)
        
    true_data = data[left]
    false_data = data[right]
    
    true_labels = labels[left]
    false_labels = labels[right]
        
    return true_data, false_data, true_labels, false_labels

<a id='11'>ячейка с изменениями 2</a> <br>
Тут я закомментировал root_gini = gini(labels) и добавил вариант через mse, а так же ближе к концу заменил вызов функции gini на mse <br>
[Мне на верх](#vverh)

In [8]:
# Нахождение наилучшего разбиения

def find_best_split(data, labels):
    
    #  обозначим минимальное количество объектов в узле
    min_samples_leaf = 3

#     root_gini = gini(labels) <<<<<<<<<<<<<<<<<<<
    root_gini = mse(labels) # <<<<<<<<<<<<<<<<<<<
    best_gain = 0
    best_t = None
    best_index = None
    
    n_features = data.shape[1]
    
    for index in range(n_features):
        # будем проверять только уникальные значения признака, исключая повторения
        t_values = np.unique(data[:, index])
        
        for t in t_values:
            true_data, false_data, true_labels, false_labels = split(data, labels, index, t)
            #  пропускаем разбиения, в которых в узле остается менее 5 объектов
            if len(true_data) < min_samples_leaf or len(false_data) < min_samples_leaf:
                continue
            
            current_gain = gain(true_labels, false_labels, root_gini, #gini  # <<<<<<<<<<<<<<<<<<<<<<<<<
                                mse                                          # <<<<<<<<<<<<<<<<<<<<<<<<<
                               )
            
            #  выбираем порог, на котором получается максимальный прирост качества
            if current_gain > best_gain:
                best_gain, best_t, best_index = current_gain, t, index

    return best_gain, best_t, best_index

[Мне на верх](#vverh)

In [9]:
import time
# Построение дерева с помощью рекурсивной функции

def build_tree(data, labels):

    gain, t, index = find_best_split(data, labels)

    #  Базовый случай - прекращаем рекурсию, когда нет прироста в качества
    if gain == 0:
        return Leaf(data, labels)

    true_data, false_data, true_labels, false_labels = split(data, labels, index, t)

    # Рекурсивно строим два поддерева
    true_branch = build_tree(true_data, true_labels)

#     print(time.time(), true_branch)
    false_branch = build_tree(false_data, false_labels)
    
#     print(time.time(), false_branch)
    
    # Возвращаем класс узла со всеми поддеревьями, то есть целого дерева
    return Node(index, t, true_branch, false_branch)

In [10]:
def classify_object(obj, node):

    #  Останавливаем рекурсию, если достигли листа
    if isinstance(node, Leaf):
        answer = node.prediction
        return answer

    if obj[node.index] <= node.t:
        return classify_object(obj, node.true_branch)
    else:
        return classify_object(obj, node.false_branch)

In [11]:
def predict(data, tree):
    
    classes = []
    for obj in data:
        prediction = classify_object(obj, tree)
        classes.append(prediction)
    return classes

In [12]:
# Напечатаем ход нашего дерева
def print_tree(node, spacing=""):

    # Если лист, то выводим его прогноз
    if isinstance(node, Leaf):
        print(spacing + "Прогноз:", node.prediction)
        return

    # Выведем значение индекса и порога на этом узле
    print(spacing + 'Индекс', str(node.index), '<=', str(node.t))

    # Рекурсионный вызов функции на положительном поддереве
    print(spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # Рекурсионный вызов функции на отрицательном поддереве
    print(spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")
    


In [13]:
# сгенерируем данные
classification_data, classification_labels = make_classification(n_features=2, n_informative=2, 
                                                                 n_classes=2, n_redundant=0,
                                                                 n_clusters_per_class=1, random_state=5)
# classification_data, classification_labels = make_circles(n_samples=50, random_state=5)

In [14]:
# Разобьем выборку на обучающую и тестовую

from sklearn.model_selection import train_test_split

train_data, test_data, train_labels, test_labels = train_test_split(classification_data, 
                                                                    classification_labels, 
                                                                    test_size=0.3,
                                                                    random_state=1)

In [15]:
df_full = pd.read_csv('./data/cardio.csv', sep=';')

features = ['age', 'height']
target = ['cardio']

df = df_full[features + target]
print(df.shape)

train_data, test_data, train_labels, test_labels = train_test_split(df[features].values, 
                                                                    np.squeeze(df[target].values),
                                                                    test_size=0.3,
                                                                    random_state=1)

(70000, 3)


In [16]:
%%time

my_tree = build_tree(train_data, train_labels)

Wall time: 991 ms


In [17]:
print_tree(my_tree)

Индекс 0 <= 54
--> True:
  Индекс 0 <= 44
  --> True:
    Индекс 0 <= 40
    --> True:
      Индекс 1 <= 159
      --> True:
        Индекс 1 <= 152
        --> True:
          Индекс 0 <= 39
          --> True:
            Индекс 1 <= 151
            --> True:
              Индекс 1 <= 146
              --> True:
                Индекс 1 <= 142
                --> True:
                  Индекс 1 <= 120
                  --> True:
                    Прогноз: 0.0
                  --> False:
                    Прогноз: 0.2
                --> False:
                  Прогноз: 0.4
              --> False:
                Индекс 1 <= 149
                --> True:
                  Прогноз: 0.0
                --> False:
                  Индекс 1 <= 150
                  --> True:
                    Прогноз: 0.07692307692307693
                  --> False:
                    Прогноз: 0.0
            --> False:
              Прогноз: 0.2222222222222222
          --> False:
           

                        --> True:
                          Прогноз: 0.5
                        --> False:
                          Индекс 1 <= 186
                          --> True:
                            Прогноз: 0.25
                          --> False:
                            Прогноз: 0.42857142857142855
                --> False:
                  Прогноз: 0.6
            --> False:
              Прогноз: 0.0
    --> False:
      Индекс 0 <= 52
      --> True:
        Индекс 1 <= 179
        --> True:
          Индекс 1 <= 149
          --> True:
            Индекс 1 <= 81
            --> True:
              Прогноз: 0.25
            --> False:
              Индекс 1 <= 120
              --> True:
                Прогноз: 1.0
              --> False:
                Индекс 1 <= 136
                --> True:
                  Прогноз: 0.0
                --> False:
                  Индекс 1 <= 143
                  --> True:
                    Прогноз: 0.8
           

              --> True:
                Индекс 1 <= 158
                --> True:
                  Прогноз: 0.6601941747572816
                --> False:
                  Прогноз: 0.5957446808510638
              --> False:
                Индекс 1 <= 158
                --> True:
                  Прогноз: 0.6901408450704225
                --> False:
                  Прогноз: 0.6976744186046512
            --> False:
              Индекс 0 <= 61
              --> True:
                Индекс 1 <= 162
                --> True:
                  Индекс 1 <= 160
                  --> True:
                    Прогноз: 0.6992481203007519
                  --> False:
                    Индекс 1 <= 161
                    --> True:
                      Прогноз: 0.5869565217391305
                    --> False:
                      Прогноз: 0.6962025316455697
                --> False:
                  Индекс 1 <= 163
                  --> True:
                    Прогноз: 0.7692307

In [18]:
# Получим ответы для обучающей выборки 
train_answers = predict(train_data, my_tree)

In [19]:
# И получим ответы для тестовой выборки
answers = predict(test_data, my_tree)

In [20]:
# Введем функцию подсчета точности как доли правильных ответов
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

In [21]:
# Точность на обучающей выборке
train_accuracy = accuracy_metric(train_labels, train_answers)
train_accuracy

0.5163265306122449

In [22]:
# Точность на тестовой выборке
test_accuracy = accuracy_metric(test_labels, answers)
test_accuracy

0.37142857142857144